In [1]:
import sys
import numpy as np
import os
import shutil
import subprocess
import time
from pathlib import Path
import yaml
import re

from pymatgen.core import Structure
from pymatgen.io.vasp.sets import MPRelaxSet
from pymatgen.io.vasp.inputs import Kpoints, Incar, Potcar
from pymatgen.symmetry.bandstructure import HighSymmKpath
from pymatgen.io.vasp.outputs import Vasprun
from pymatgen.electronic_structure.plotter import BSPlotter
import matplotlib.pyplot as plt

In [2]:
def submit_job(script_path):
    """Submit the job to the queue."""
    try:
        result = subprocess.run(['qsub', script_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
        if result.returncode != 0:
            raise RuntimeError(f"Job submission failed: {result.stderr.decode().strip()}")
        # Extract job ID from the output
        job_id = result.stdout.decode().strip()
        return job_id
    except Exception as e:
        print(f"Error during job submission: {e}")
        raise


def is_job_running(job_id):
    """Check if the job is still running."""
    result = subprocess.run(['qstat', job_id], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode == 0:
        return True  # Job is still running
    elif "Unknown Job Id" in result.stderr:
        return False  # Job is not running (completed)
    else:
        raise RuntimeError(f"Error checking job status: {result.stderr}")


def wait_for_job_completion(job_id, poll_interval=30):
    """Wait until the job with job_id is completed."""
    while is_job_running(job_id):
        print(f"Job {job_id} is still running. Checking again in {poll_interval} seconds.")
        time.sleep(poll_interval)
    print(f"Job {job_id} has completed.")


def prepare_directory(directory):
    """Create directory if it doesn't exist."""
    Path(directory).mkdir(parents=True, exist_ok=True)


def run_vasp_calculation(job_directory, script_path):
    """Submit VASP job and wait for completion."""
    os.chdir(job_directory)
    try:
        job_id = submit_job(script_path)
        print(f"Submitted job with ID: {job_id}")
        wait_for_job_completion(job_id)
        print("Job completed. Continuing with the next step...")
    finally:
        os.chdir(original_directory)

# check what is ibz here
def setup_kpoints_hsp():
    """Setup band structure calculations."""
    structure = Structure.from_file(f"{relax}/POSCAR")
    kpath = HighSymmKpath(structure)
    kpts = Kpoints.automatic_linemode(divisions=40, ibz=kpath)
    kpts.write_file(os.path.join(f"{relax}/", "KPOINTS_hsp"))


def modify_incar_from_dict(incar_file_path, custom_settings):
    """Modify INCAR file based on custom settings dictionary."""
    # Read the existing INCAR file content
    with open(incar_file_path, 'r') as file:
        lines = file.readlines()

    # Flag to check if each key was found
    settings_found = {key: False for key in custom_settings}

    # Modify or add entries from the dictionary
    with open(incar_file_path, 'w') as file:
        for line in lines:
            # Check if the line contains any of the keys in the dictionary (even if commented)
            for key in custom_settings:
                if re.search(rf'^[!\s]*{key}\s*=', line):
                    settings_found[key] = True
                    # Comment the existing line
                    line = re.sub(rf'^[!\s]*({key}\s*=.*)', rf'!\1', line)
            file.write(line)

        # Add or update the settings based on the dictionary
        for key, value in custom_settings.items():
            if not settings_found[key]:
                file.write(f'{key} = {value}\n')
            else:
                # If the key was found, write the new value
                file.write(f'{key} = {value}\n')



# here file_path = POTCAR file
# here file_path is of POTCAR
def modify_encut(incar_file_path, file_path):
    """Modify ENCUT file with custom settings."""
    
    max_enmax = None

    with open(file_path, 'r') as file:
        for line in file:
            if 'ENMAX' in line:
                enmax_str = line.split('=')[1].split(';')[0].strip()
            
                try:
                    enmax_value = float(enmax_str)
                except ValueError:
                    continue  # Skip this line if conversion fails
            
                if max_enmax is None or enmax_value > max_enmax:
                    max_enmax = enmax_value

    if max_enmax is None:
        raise ValueError("No valid ENMAX value found in the provided file.")

    encut = int(1.5 * max_enmax)  # Adjust factor (1.3 or 1.5) as needed
    
    # Read the existing INCAR file content
    with open(incar_file_path, 'r') as file:
        lines = file.readlines()

    # Flag to check if ENCUT was found
    encut_found = False

    # Modify or add ENCUT
    with open(incar_file_path, 'w') as file:
        for line in lines:
            # Check for ENCUT in the line (even if commented)
            if re.search(r'^[!\s]*ENCUT\s*=', line):
                encut_found = True
                # Uncomment if necessary and update the value
                line = re.sub(r'^[!\s]*ENCUT\s*=.*', f'ENCUT = {encut}', line)
            file.write(line)
        
        # If ENCUT was not found, add it to the end
        if not encut_found:
            file.write(f'\nENCUT = {encut}\n')


# here filename will be cif_file
def extract_information(filename):
    database_code_pattern = r'^#_database_code_PCD\s+(\d+)'
    formula_pattern = r"_chemical_formula_structural\s+(\S+)"

    database_code = None
    formula = None

    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading and trailing whitespace

            # Extract database code
            match_db_code = re.match(database_code_pattern, line)
            if match_db_code:
                database_code = match_db_code.group(1)
            
            # Extract chemical formula
            match_formula = re.search(formula_pattern, line)
            if match_formula:
                formula = match_formula.group(1)
                # If formula was found, break out of the loop
                break
    
    # If formula is still None, raise an error or handle as needed
    if formula is None:
        raise ValueError("Chemical formula not found in the file.")
    
    # Clean up the formula to create the system name
    # system_name = re.sub(r'[^\w\s]', '', formula)
    # system_name = system_name.replace(' ', '')
    system_name = formula
    # If database code is None, raise an error or handle as needed
    if database_code is None:
        raise ValueError("Database code not found in the file.")
    
    return database_code, system_name


def extract_nelect(file_path):
    nelect_value = None

    pattern = re.compile(r'NELECT\s*=\s*([\d.]+)')

    with open(file_path, 'r') as file:
        for line in file:
            match = pattern.search(line)
            if match:
                nelect_value = int(float(match.group(1)))
                break

    return nelect_value


def modify_nbands_wsoc(incar_file_path, outcar_file_path):
    """Modify NBANDS for wsoc file with custom settings."""
    nelect = extract_nelect(outcar_file_path)
    nbands = int(int(((int(nelect / 2) + 16) / 32) + 1) * 32)

    # Read the existing INCAR file content
    with open(incar_file_path, 'r') as file:
        lines = file.readlines()

    # Flag to check if NBANDS was found
    nbands_found = False

    # Modify or add NBANDS
    with open(incar_file_path, 'w') as file:
        for line in lines:
            # Check for NBANDS in the line (even if commented)
            if re.search(r'^[!\s]*NBANDS\s*=', line):
                nbands_found = True
                # Comment the existing NBANDS line
                line = re.sub(r'^[!\s]*(NBANDS\s*=.*)', r'!\1', line)
            file.write(line)
        
        # Add the new NBANDS value at the end
        file.write(f'NBANDS = {nbands}\n')



def modify_nbands_soc(incar_file_path, outcar_file_path):
    """Modify NBANDS for soc file with custom settings."""
    nelect = extract_nelect(outcar_file_path)
    nbands = int(int(((int(nelect) + 16) / 32) + 1) * 32)

    # Read the existing INCAR file content
    with open(incar_file_path, 'r') as file:
        lines = file.readlines()

    # Flag to check if NBANDS was found
    nbands_found = False

    # Modify or add NBANDS
    with open(incar_file_path, 'w') as file:
        for line in lines:
            # Check for NBANDS in the line (even if commented)
            if re.search(r'^[!\s]*NBANDS\s*=', line):
                nbands_found = True
                # Comment the existing NBANDS line
                line = re.sub(r'^[!\s]*(NBANDS\s*=.*)', r'!\1', line)
            file.write(line)
        
        # Add the new NBANDS value at the end
        file.write(f'NBANDS = {nbands}\n')

# As of now not required
def modify_magmom(incar_file_path):
    """Modify MAGMOM file with custom settings."""
    incar = Incar.from_file(incar_file_path)
    #incar.update(custom_settings)
    structure = Structure.from_file(f"{soc_scf}/POSCAR")
    magmom = [0.0] * len(structure)  # Example: setting MAGMOM for all sites to 5.0 mu_B
    custom_settings = {"MAGMOM": magmom}
    incar.write_file(incar_file_path)


    relax = Path(f"../calculations/{main_directory}/relax")
def plot_band_structure(vasprun_file, output_file):
    """Plot band structure."""
    vaspout = Vasprun(vasprun_file)
    bandstr = vaspout.get_band_structure(line_mode=True)
    bs_plotter = BSPlotter(bandstr)
    plot = bs_plotter.get_plot(zero_to_efermi=True, ylim=[-2, 2])
    plt.axhline(y=0, color='k', linestyle='--', linewidth=1, label="Fermi Level")
    plt.legend().set_visible(False)
    plt.savefig(output_file)
    plt.close()

In [4]:

    cif_file = "1951087.cif"
    
    original_directory = Path.cwd()

    database_code, system_name = extract_information(cif_file)
    print(f"Database code: {database_code}, System name: {system_name}")
    
    #extract_information(cif_file)
    
    main_directory = f"{database_code}_{system_name}"
    
    relax = Path(f"../calculations/{main_directory}/relax")
    wsoc_scf = Path(f"../calculations/{main_directory}/bands/wsoc/scf")
    wsoc_dos = Path(f"../calculations/{main_directory}/bands/wsoc/dos")
    wsoc_band = Path(f"../calculations/{main_directory}/bands/wsoc/band")
    wsoc_plots = Path(f"../calculations/{main_directory}/bands/wsoc/plots")

    soc_scf = Path(f"../calculations/{main_directory}/bands/soc/scf")
    soc_dos = Path(f"../calculations/{main_directory}/bands/soc/dos")
    soc_band = Path(f"../calculations/{main_directory}/bands/soc/band")
    soc_plots = Path(f"../calculations/{main_directory}/bands/soc/plots")

    job_std = Path("../job_files/job_std.sh")
    job_ncl = Path("../job_files/job_ncl.sh")

    # Step 1: Relaxation
    struct = Structure.from_file(filename=cif_file)
    # user_incar_params = ordered_yaml_load('my_incar.yaml')
    relax_set = MPRelaxSet(structure=struct)
    relax_set.write_input(output_dir=f"{relax}")
    # Bring job_std file
    # shutil.copy(job_std, relax)
    
    # For incar_files
    shutil.copy("../incar_files/INCAR_relax", f"{relax}/INCAR")

    # need to be changed
    modify_encut(f"{relax}/INCAR", f"{relax}/POTCAR")

    #TODO
    # Check what is ibz inside this function.
    setup_kpoints_hsp()


Database code: 1951087, System name: Si


/home/rudra/.conda/envs/dft/lib/python3.12/site-packages/pymatgen/symmetry/kpath.py:160: UserWarning: The input structure does not match the expected standard primitive! The path may be incorrect. Use at your own risk.
  warn(


In [1]:
import os
import shutil

In [2]:
cif_file = "1951087.cif"

In [4]:
shutil.copy(cif_file, "../calculated/")


FileNotFoundError: [Errno 2] No such file or directory: 'calculated/1951087.cif' -> 'database'

In [5]:
def rename_file(directory, old_filename, new_filename):
    # Construct the full file path for the old and new filenames
    old_file_path = os.path.join(directory, old_filename)
    new_file_path = os.path.join(directory, new_filename)

    # Rename the file
    os.rename(old_file_path, new_file_path)

In [6]:
# Example usage
directory = "../calculated/"
old_filename = cif_file
new_filename = 'database'

rename_file(directory, old_filename, new_filename)